In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.optimize import fminbound
from math import pi, sqrt, atan, atan2, tan, cos, sin, radians, degrees

from chartInterpolation import dual_target_residual, AM_1stChartInterpolation, find_min_ypa_for_angle_inAM1stChart
from meanlinePlot import stator_2D_meanline_plot, rotor_2D_meanline_plot

# Combustion Model

In [ ]:
def combustion_model_forProject(m_dot_air, m_dot_fuel):
    M_fuel = 167.3e-3  # kg/mol, Jet-A1 approx. C12H23
    M_air = 28.85e-3   # kg/mol (average)

    n_fuel = m_dot_fuel / M_fuel
    n_air = m_dot_air / M_air

    n_O2_st = 17.75
    mass_air_st_per_mol_fuel = n_O2_st * (1 + 3.727 + 0.0444) * M_air
    AFR_st = mass_air_st_per_mol_fuel / M_fuel
    AFR_actual = m_dot_air / m_dot_fuel
    lambda_ = AFR_actual / AFR_st

    n_O2 = lambda_ * n_O2_st * n_fuel
    n_N2 = n_O2 * 3.727
    n_Ar = n_O2 * 0.0444

    n_CO2 = 12 * n_fuel
    n_H2O = 11.5 * n_fuel
    n_O2_prod = n_O2 - n_O2_st * n_fuel

    M = {"CO2": 0.04401, "H2O": 0.01802, "O2": 0.03200, "N2": 0.02802, "Ar": 0.03995}

    products = {"CO2": n_CO2, "H2O": n_H2O, "O2": n_O2_prod, "N2": n_N2, "Ar": n_Ar}
    total_moles = sum(products.values())
    mole_fractions = {gas: n / total_moles for gas, n in products.items()}
    mass_total = sum(products[gas] * M[gas] for gas in products)
    mass_fractions = {gas: (products[gas] * M[gas]) / mass_total for gas in products}

    return mole_fractions, mass_fractions


# ----------------------------
# INPUT DATA SECTION
# ----------------------------
T3 = 1700  # Temperature in Kelvin
R = 8.314510  # J/mol.K
m_dot = 250.0  # kg/s
BPR = 0.68
cooling_air = 0.1
m_dot_fuel = 2.18  # kg/s
m_dot_core = m_dot / (1+BPR) * (1-cooling_air)
m_dot_bypassed = m_dot-m_dot_core

# Gas data: molar mass [kg/mol] and NASA coefficients [a1-a5]
gas_data = {
    "CO2": {"MW": 0.04401, "coeffs": [4.63659493, 2.74131991e-3, -9.95828531e-7, 1.60373011e-10, -9.16103468e-15]},
    "H2O": {"MW": 0.018015, "coeffs": [2.67703787, 2.97318329e-3, -7.73769690e-7, 9.44336689e-11, -4.26900959e-15]},
    "O2":  {"MW": 0.03200, "coeffs": [3.66096083, 6.56365523e-4, -1.41149485e-7, 2.05797658e-11, -1.29913248e-15]},
    "N2":  {"MW": 0.028013, "coeffs": [3.11567530, 1.45886880e-3, -6.01731480e-7, 1.13484230e-10, -7.96585180e-15]},
    "Ar":  {"MW": 0.03995, "coeffs": [2.5, 0, 0, 0, 0]}
}


# ----------------------------
# MAIN PROPERTY CALCULATION
# ----------------------------
mole_fractions, mass_fractions = combustion_model_forProject(m_dot_core, m_dot_fuel)

cp_mix = cv_mix = 0.0
rows = []

for gas, props in gas_data.items():
    a = props["coeffs"]
    MW = props["MW"]
    cp_R = sum(a[i] * T3**i for i in range(5))
    cp = (cp_R * R / MW) / 1000  # kJ/kg.K
    R_i = R / MW / 1000  # kJ/kg.K
    cv = cp - R_i
    gamma = cp / cv

    cp_mix += mass_fractions[gas] * cp
    cv_mix += mass_fractions[gas] * cv

    rows.append({"Gas": gas, "cp [kJ/kg.K]": cp, "cv [kJ/kg.K]": cv, "gamma": gamma, "R [kJ/kg.K]": R_i})

M_mix = sum(mole_fractions[gas] * gas_data[gas]["MW"] for gas in gas_data)
R_mix = R / M_mix / 1000
gamma_mix = cp_mix / cv_mix

rows.append({"Gas": "Mixture", "cp [kJ/kg.K]": cp_mix, "cv [kJ/kg.K]": cv_mix, "gamma": gamma_mix, "R [kJ/kg.K]": R_mix})


# ----------------------------
# VISCOSITY CALCULATION
# ----------------------------
viscosity_file = "data/TURBOMACHINERY Design project-GasViscosityDatabase.xlsx"
df_mu = pd.read_excel(viscosity_file)

target_gases = ["CO2", "H2O", "O2", "N2", "AR"]
mu0, T0, S = {}, {}, {}

for gas in target_gases:
    row = df_mu[df_mu["Fluido"].str.upper() == gas]
    mu0[gas] = float(row["mu0"])
    T0[gas] = float(row["T0mu"])
    S[gas] = float(row["Smu"])

mu = {}
for gas in target_gases:
    mu[gas] = mu0[gas] * (T3 / T0[gas])**1.5 * (T0[gas] + S[gas]) / (T3 + S[gas])

molar_weights = {"CO2": 0.04401, "H2O": 0.018015, "O2": 0.032, "N2": 0.028013, "AR": 0.03995}
mol_fracs = {"CO2": mole_fractions["CO2"], "H2O": mole_fractions["H2O"], "O2": mole_fractions["O2"], "N2": mole_fractions["N2"], "AR": mole_fractions["Ar"]}

phi = {}
for i in target_gases:
    phi[i] = {}
    for j in target_gases:
        if i == j:
            phi[i][j] = 1
        else:
            mu_i = mu[i]
            mu_j = mu[j]
            Mi = molar_weights[i]
            Mj = molar_weights[j]
            phi[i][j] = (1 + (mu_i / mu_j)**0.5 * (Mj / Mi)**0.25)**2 / (8 * (1 + Mi / Mj))**0.5

mu_mix = 0
for i in target_gases:
    numerator = mol_fracs[i] * mu[i]
    denominator = sum(mol_fracs[j] * phi[i][j] for j in target_gases)
    mu_mix += numerator / denominator

# ----------------------------
# EXPORT TO CSV
# ----------------------------
df = pd.DataFrame(rows)
df.loc[df.shape[0]] = {"Gas": "Viscosity [Pa.s]", "cp [kJ/kg.K]": mu_mix}
df.loc[df.shape[0]] = {"Gas": "m_dot_core [kg/s]", "cp [kJ/kg.K]": m_dot_core}
df.loc[df.shape[0]] = {"Gas": "m_dot_fuel [kg/s]", "cp [kJ/kg.K]": m_dot_fuel}
df.to_csv("data/FG_flue_gas_properties_1700K.csv", index=False)
print(df)

# Turbine Stage Design

## Meanline
3 stations + outer $\eta$ loop

In [ ]:
# === Section 1: Initialization and Inputs ===

data_dir = 'data'
os.makedirs(data_dir, exist_ok=True)

# Load gas properties from CSV
gas_props = pd.read_csv(os.path.join(data_dir, 'FG_flue_gas_properties_1700K.csv'))
props = {col: gas_props[col].values[5] for col in gas_props.columns}

cp_mixture = props['cp [kJ/kg.K]']            # [kJ/kg-K]
gamma_mixture = props['gamma']                # [-]
R_mix = props['R [kJ/kg.K]']                  # [kJ/kg-K]

# You may need to add these if not in the CSV
mu_mix = gas_props.loc[gas_props['Gas'] == 'Viscosity [Pa.s]', 'cp [kJ/kg.K]'].values[0]
mass_flow_air = gas_props.loc[gas_props['Gas'] == 'm_dot_core [kg/s]', 'cp [kJ/kg.K]'].values[0]
mass_flow_fuel = gas_props.loc[gas_props['Gas'] == 'm_dot_fuel [kg/s]', 'cp [kJ/kg.K]'].values[0]
T3 = 1700         # [K]

# Input parameters
cooling_air = 0.10
T0t = T3
P0t = 40  # bar
ExpRatio_tot = 2
P2t = P0t / ExpRatio_tot

mass_flow_total = mass_flow_air / (1 - cooling_air) + mass_flow_fuel

# Geometry assumptions
clearance = 0.5e-3  # m
chord_stator = 0.04  # m
chord_rotor = 0.04   # m
alfa0 = 0  # deg
omega = 15000  # RPM

# Non-dimensional parameters
work_coeff = 1.45   # [Psi]
flow_coeff = 0.58   # [phi]
eta_tt = 94.1 / 100  #
alfa2 = 0  # deg (no swirl at rotor exit) -> V2tang = 0

# Save key inputs
input_dict = {
    'T0t': T0t, 'P0t': P0t, 'P2t': P2t,
    'mass_flow_total': mass_flow_total,
    'omega': omega,
    'work_coeff': work_coeff, 'flow_coeff': flow_coeff, 'eta_tt': eta_tt,
    'cp_mixture': cp_mixture, 'gamma_mixture': gamma_mixture, 'R_mix': R_mix
}
pd.Series(input_dict).to_csv(os.path.join(data_dir, 'TM_turbine_inputs.csv'))

In [ ]:
# === Section 2: Station 1 (Rotor Inlet) ===

# Isentropic and actual work
L_is = cp_mixture * T0t * (1 - (P2t / P0t)**((gamma_mixture - 1) / gamma_mixture))
L_euler = eta_tt * L_is

# Mean blade speed and radius
U_mean = sqrt(L_euler * 1000 / work_coeff)
omega_rad = 2 * pi * omega / 60
r_mean = U_mean / omega_rad

# Velocity triangles
V1ax = flow_coeff * U_mean
alfa1 = degrees(atan(work_coeff / flow_coeff))
V1tang = tan(radians(alfa1)) * V1ax
V1 = sqrt(V1ax**2 + V1tang**2)
W1ax = V1ax
W1tang = V1tang - U_mean
W1 = sqrt(W1ax**2 + W1tang**2)
beta1 = degrees(atan(W1tang / W1ax))

# Temperatures and Mach numbers
T1t = T0t
T1 = T1t - (V1**2 / (2 * cp_mixture * 1000))
M1 = V1 / sqrt(gamma_mixture * R_mix * 1000 * T1)
M1rel = W1 / sqrt(gamma_mixture * R_mix * 1000 * T1)
T1trel = T1 * (1 + (gamma_mixture - 1) / 2 * M1rel**2)

# Save station 1 results
station1_dict = {
    'L_is': L_is,
    'L_euler': L_euler,
    'U_mean': U_mean,
    'r_mean': r_mean,
    'omega_rad': omega_rad,
    'V1ax': V1ax,
    'V1tang': V1tang,
    'V1': V1,
    'W1ax': W1ax,
    'W1tang': W1tang,
    'W1': W1,
    'alfa1': alfa1,
    'beta1': beta1,
    'T1t': T1t,
    'T1': T1,
    'M1': M1,
    'M1rel': M1rel,
    'T1trel': T1trel
}
pd.Series(station1_dict).to_csv(os.path.join(data_dir, 'TM_station1_results.csv'))

In [ ]:
# === Section 3: Station 1 Losses with Interpolation ===

def station1_losses_with_interpolation(params):
    import numpy as np
    import pandas as pd
    import os

    rho1_guess = params['P0t'] * 1e5 / (params['R_mix'] * 1000 * params['T0t'])
    #print(params['P0t'], params['R_mix'], params['T0t'])
    #print(f'rho1_guess: {rho1_guess}')
    b1_old = params['mass_flow_total'] / (rho1_guess * params['V1'] * 2 * np.pi * params['r_mean'])

    tol = 1e-10
    error = 1

    results = []

    while error > tol:
        # === Yp from Ainley-Mathieson Min Point ===
        closest_beta, s_c_min, yp_min = find_min_ypa_for_angle_inAM1stChart(
            params['filename_AM1'], params['alfa1']
        )
        pitch = s_c_min * params['chord_stator']
        num_blades = round((2 * np.pi * params['r_mean']) / pitch)
        pitch_new = 2 * np.pi * params['r_mean'] / num_blades
        s2c = pitch_new / params['chord_stator']

        Yp_ref = AM_1stChartInterpolation(params['alfa1'], s2c, params['filename_AM1'])
        Xi = 1.0
        M1 = params['M1']
        Y_profile = (1 + 60 * (M1 - 1)**2) * Xi * Yp_ref

        alfa_mean = np.degrees(np.arctan((np.tan(np.radians(params['alfa0'])) + np.tan(np.radians(params['alfa1']))) / 2))
        C_lift = 2 * s2c * abs(np.tan(np.radians(params['alfa0']) - np.radians(params['alfa1']))) * np.cos(np.radians(alfa_mean))
        Y_secondary = (params['chord_stator'] / b1_old) * 0.0334 * (np.cos(np.radians(params['alfa1'])) / np.cos(np.radians(params['alfa0']))) * \
                      (C_lift / s2c)**2 * (np.cos(np.radians(params['alfa1'])))**2 / (np.cos(np.radians(alfa_mean)))**3

        Re = rho1_guess * params['V1'] * params['chord_stator'] / params['mu_mix']
        X_Re = (Re / 2e5)**(-0.2)

        t_trailing = 0.5e-3
        t2s_ref = 0.02
        X_TE = 1 + 7 * ((t_trailing / pitch_new) - t2s_ref)

        Y_total = Y_profile * X_Re * X_TE + Y_secondary

        gamma = params['gamma_mixture']
        t2s_ratio = (1 + (gamma - 1)/2 * M1**2)**(gamma / (gamma - 1))
        P1t = params['P0t'] / (1 + Y_total - (Y_total / t2s_ratio))
        P1 = P1t / t2s_ratio

        rho1_new = P1 * 1e5 / (params['R_mix'] * 1000 * params['T1'])
        b1_new = params['mass_flow_total'] / (rho1_new * params['V1ax'] * 2 * np.pi * params['r_mean'])

        error = abs(b1_new - b1_old) / b1_old
        b1_old = b1_new
        rho1_guess = rho1_new

        results.append([Y_total, rho1_new, b1_new, error])

    final = {
        'Y_total_stator': Y_total,
        'Y_secondary_calculated': Y_secondary,
        'P1t': P1t,
        'P1': P1,
        'rho1': rho1_new,
        'b1': b1_new
    }
    pd.DataFrame(results, columns=['Y_total', 'rho1', 'b1', 'error']).to_csv(os.path.join('data', 'TM_station1_iterations.csv'), index=False)
    return final



# Prepare inputs for interpolation function
station1_params = {
    'P0t': P0t,
    'T0t': T0t,
    'T1': T1,
    'P0': P0t,
    'alfa0': alfa0,
    'alfa1': alfa1,
    'V1': V1,
    'V1ax': V1ax,
    'M1': M1,
    'mass_flow_total': mass_flow_total,
    'r_mean': r_mean,
    'R_mix': R_mix,
    'cp_mixture': cp_mixture,
    'gamma_mixture': gamma_mixture,
    'mu_mix': mu_mix,
    'chord_stator': chord_stator,
    'filename_AM1': os.path.join('data', 'AM_1st_chart_interpolation_Points.txt')
}

# Run iterative station 1 loss calculation with interpolation
station1_losses = station1_losses_with_interpolation(station1_params)

# Save final results and history
pd.Series(station1_losses).to_csv(os.path.join(data_dir, 'TM_station1_losses.csv'))

print("Station 1 loss loop complete.")
print(f"Final b1 = {station1_losses['b1']:.5f} m, P1 = {station1_losses['P1']:.3f} bar")

# Store b1 and rho1 for downstream calculations
b1_new = station1_losses['b1']
Rho1_new = station1_losses['rho1']
P1t = station1_losses['P1t']
P1 = station1_losses['P1']

In [ ]:
# === Section 4: Station 0 (Stator Inlet) ===

# Initial guess and reused variables
rho0_guess = Rho1_new  # from Station 1 final result
b0 = b1_new             # assume constant blade height
V0_old = mass_flow_total / (rho0_guess * 2 * pi * r_mean * b0)

tolerance = 1e-6
error = 1.0
iteration = 0

station0_log = []

while error > tolerance:
    iteration += 1

    # Temperature from energy balance
    T0 = T0t - V0_old**2 / (2 * cp_mixture * 1000)
    M0 = V0_old / sqrt(gamma_mixture * R_mix * 1000 * T0)
    P0 = P0t / (1 + (gamma_mixture - 1) * 0.5 * M0**2)**(gamma_mixture / (gamma_mixture - 1))
    rho0_new = P0 * 1e5 / (R_mix * 1000 * T0)

    V0_new = mass_flow_total / (rho0_new * 2 * pi * r_mean * b0)
    error = abs((V0_new - V0_old) / V0_old)

    station0_log.append([iteration, V0_new, T0, M0, P0, rho0_new, error])

    V0_old = V0_new

# Save final Station 0 values
station0_data = {
    'V0': V0_new,
    'T0': T0,
    'M0': M0,
    'P0': P0,
    'rho0': rho0_new,
    'T0t': T0t,
    'P0t': P0t
}
pd.Series(station0_data).to_csv(os.path.join(data_dir, 'TM_station0_results.csv'))

# Save iteration history
station0_df = pd.DataFrame(station0_log, columns=[
    'Iteration', 'V0', 'T0', 'M0', 'P0', 'rho0', 'Error'])
station0_df.to_csv(os.path.join(data_dir, 'TM_station0_iterations.csv'), index=False)

In [ ]:
# === Section 5: Station 2 (Rotor Outlet Optimization) Refactored ===

# Compute relative temperature and pressures at station 1
T2t = T1t - L_euler / cp_mixture
T1rel = T1t - W1**2 / (2 * cp_mixture * 1000)
P1trel = P1 * (1 + ((gamma_mixture - 1) / 2) * M1rel**2) ** (gamma_mixture / (gamma_mixture - 1))
P1rel = P1trel * (T1rel / T1trel) ** (gamma_mixture / (gamma_mixture - 1))

# Assemble all necessary parameters for rotor outlet loss model
params = {
    'T1t': T1t,
    'L_euler': L_euler,
    'cp_mixture': cp_mixture,
    'gamma_mixture': gamma_mixture,
    'R_mix': R_mix,
    'P2t': P2t,
    'r_mean': r_mean,
    'b1_new': b1_new,
    'alfa0': alfa0,
    'U_mean': U_mean,
    'mu_mix': mu_mix,
    'mass_flow_total': mass_flow_total,
    'chord_rotor': chord_rotor,
    'beta1': beta1,
    'M1': M1,
    'M1rel': M1rel,
    'P1trel': P1trel,
    'P1rel': P1rel,
    'te_thickness_rotor': chord_rotor * 0.025,
    'throat_op_rotor': chord_rotor * 0.2,
    'numSeals': 3,
    'tipClearance': 0.5e-3,
    'maxBladeThickness_rotor': 0.01,
    'filename_AM1': os.path.join(data_dir, 'AM_1st_chart_interpolation_Points.txt'),
    'filename_AM2': os.path.join(data_dir, 'AM_2nd_chart_interpolation_Points.txt'),
    'filename_AM3': os.path.join(data_dir, 'ainley mathieson 3rd graph interpolation points.txt'),
    'filename_rotor_shock_losses': os.path.join(data_dir, 'rh2rtVSM1hub2M1_KackerOcapuu_Rotor.txt'),
    'filename_tet': os.path.join(data_dir, 't2oVsTEEC.txt'),
    'r_hub': 0.180
}

In [ ]:
# === Section 6: Solve Rotor Outlet Velocity with Dual Target Residual ===

print("\nStarting optimization for rotor outlet axial velocity V2...")
V2_opt = fminbound(lambda V2: dual_target_residual(V2, params), 160.9, 526.7)

# Load computed losses and velocities from CSV output generated by residual function
loss_file = os.path.join(data_dir, 'TM_station2_rotor_losses.csv')
station2_rotor = pd.read_csv(loss_file, index_col=0).squeeze()

# Final values to export and report
station2_data = {
    'V2_opt': V2_opt,
    'T2t': T2t,
    'V2ax': station2_rotor['V2ax'],
    'V2tang': station2_rotor['V2tang'],
    'b2': station2_rotor['b2'],
    'Y_total_rotor': station2_rotor['Y_total_rotor'],
    'Y_secondary': station2_rotor['Y_secondary'],
    'Y_TC': station2_rotor['Y_TC'],
    'rho2': station2_rotor['rho2']
}

pd.Series(station2_data).to_csv(os.path.join(data_dir, 'TM_station2_results.csv'))

print("Rotor outlet optimization complete.")
print(f"Optimal axial velocity V2 = {V2_opt:.3f} m/s")

In [ ]:
# === Section 7: Blade Geometry Generation ===

import matplotlib.pyplot as plt
from scipy.optimize import fminbound
from meanlinePlot import stator_2D_meanline_plot, rotor_2D_meanline_plot

# Geometry profile parameters
x_perc = np.array([0, 1.25, 2.5, 5, 7.5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 90, 95, 98, 100])
t_perc = np.array([0, 1.124, 1.571, 2.222, 2.709, 3.111, 3.746, 4.218,
                   4.824, 5.057, 4.87, 4.151, 3.038, 1.847,
                   0.749, 0.354, 0.20, 0.15])

# Stator Geometry
max_thickness = 0.01
TE_target = 0.001
chord_target = 0.04

stator_error_func = lambda axial_chord: stator_2D_meanline_plot(
    axial_chord, max_thickness, TE_target, alfa0, alfa1, x_perc, t_perc, chord_target)[0]

axial_chord_stator = fminbound(stator_error_func, 0.01, 0.2, xtol=1e-10)
_, x_camber_s, y_camber_s, x_ss_s, y_ss_s, x_ps_s, y_ps_s, t_s = stator_2D_meanline_plot(
    axial_chord_stator, max_thickness, TE_target, alfa0, alfa1, x_perc, t_perc, chord_target)

# Rotor Geometry
beta2 = station2_rotor['beta2']
rotor_error_func = lambda axial_chord: rotor_2D_meanline_plot(
    axial_chord, max_thickness, TE_target, beta1, beta2,
    x_perc, t_perc, chord_target
)[0]

axial_chord_rotor = fminbound(rotor_error_func, 0.01, 0.2, xtol=1e-10)
_, x_camber_r, y_camber_r, x_ss_r, y_ss_r, x_ps_r, y_ps_r, t_r = rotor_2D_meanline_plot(
    axial_chord_rotor, max_thickness, TE_target, beta1, beta2, x_perc, t_perc, chord_target)

# === Plotting Profiles ===
os.makedirs('plots', exist_ok=True)

# Stator plot
plt.figure(figsize=(8,4))
plt.plot(x_ss_s, y_ss_s, 'b-', label='Suction Side')
plt.plot(x_ps_s, y_ps_s, 'r-', label='Pressure Side')
plt.plot(x_camber_s, y_camber_s, 'k--', label='Camber')
plt.axis('equal')
plt.title('Stator Blade Geometry')
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.legend()
plt.grid(True)
plt.savefig('plots/TM_stator_blade_geometry.png')

# Rotor plot
plt.figure(figsize=(8,4))
plt.plot(x_ss_r, y_ss_r, 'b-', label='Suction Side')
plt.plot(x_ps_r, y_ps_r, 'r-', label='Pressure Side')
plt.plot(x_camber_r, y_camber_r, 'k--', label='Camber')
plt.axis('equal')
plt.title('Rotor Blade Geometry')
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.legend()
plt.grid(True)
plt.savefig('plots/TM_rotor_blade_geometry.png')

print("Stator and rotor geometries generated and saved to 'plots' folder.")

In [ ]:
# === Section 8: Final Data Export and Performance Summary ===

summary = {
    'omega (RPM)': omega,
    'L_is (kJ/kg)': station1_dict['L_is'],
    'L_euler (kJ/kg)': station1_dict['L_euler'],
    'U_mean (m/s)': station1_dict['U_mean'],
    'r_mean (m)': station1_dict['r_mean'],
    'b1 (m)': b1_new,
    'V0 (m/s)': V0_new,
    'T0 (K)': T0,
    'P0 (bar)': P0,
    'rho0 (kg/m^3)': rho0_new,
    'V1 (m/s)': station1_dict['V1'],
    'V1ax (m/s)': station1_dict['V1ax'],
    'V1tang (m/s)': station1_dict['V1tang'],
    'T1 (K)': station1_dict['T1'],
    'T1t (K)': station1_dict['T1t'],
    'P1 (bar)': station1_losses['P1'],
    'rho1 (kg/m^3)': station1_losses['rho1'],
    'Y_total_stator': station1_losses['Y_total_stator'],
    'beta1 (deg)': station1_dict['beta1'],
    'alfa0 (deg)': alfa0,
    'V2opt (m/s)': station2_data['V2_opt'],
    'V2ax (m/s)': station2_data['V2ax'],
    'V2tang (m/s)': station2_data['V2tang'],
    'b2 (m)': station2_data['b2'],
    'T2t (K)': station2_data['T2t'],
    'rho2 (kg/m^3)': station2_data['rho2'],
    'Y_total_rotor': station2_data['Y_total_rotor'],
    'Y_secondary_rotor': station2_data['Y_secondary'],
    'Y_Secondary_calculated': station1_losses['Y_secondary_calculated'],
    'Y_TC': station2_data['Y_TC'],
    'T0t (K)': T0t,
    'P0t (bar)': P0t,
    'P2t (bar)': P2t,
    'Expansion Ratio': P0t / P2t,
    'Mass Flow Total (kg/s)': mass_flow_total,
    'Work Coeff (ψ)': work_coeff,
    'Flow Coeff (φ)': flow_coeff,
    'Efficiency (η_tt)': eta_tt
}

# Export to CSV
summary_path = os.path.join(data_dir, 'TM_performance_summary.csv')
pd.Series(summary).to_csv(summary_path)

print(f"\nPerformance summary written to: {summary_path}")
print("\nGenerated files:")
for file in sorted(os.listdir(data_dir)):
    print("-", file)
for file in sorted(os.listdir('plots')):
    print("- plots/", file)

## Radial Equilibrium

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import os
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

from chartInterpolation import dual_target_residual, AM_1stChartInterpolation, find_min_ypa_for_angle_inAM1stChart
from meanlinePlot import stator_2D_meanline_plot, rotor_2D_meanline_plot
from lossProfiles import secondaryLossProfile, scaleLossProfile, tipClearanceLossProfile

In [ ]:
# Radial Equilibrium - Part 1: Initialization and Station 0

# Ensure 'data' folder exists
os.makedirs('data', exist_ok=True)
data_dir = 'data'
plot_dir = 'plots'

# Load initial performance summary
perf_df = pd.read_csv('data/TM_performance_summary.csv', header=None)
perf_df[0] = perf_df[0].astype(str).str.strip()  # Ensure keys are strings
perf_dict = dict(zip(perf_df[0], perf_df[1]))

# Helper to access keys flexibly
def get_value(key_hint):
    for key in perf_dict:
        if key_hint.lower() in str(key).lower():
            return float(perf_dict[key])
    raise KeyError(f"'{key_hint}' not found in TM_performance_summary.csv")

# Extract necessary variables
r_mean = get_value('r_mean')
b1_new = get_value('b1')
omega = get_value('omega') * 2 * np.pi / 60
#print('omega = ',omega)
T0 = get_value('T0')
P0 = get_value('P0')
P0t = get_value('P0t')
alfa0 = get_value('alfa0')


# Load gas properties from CSV
gas_props = pd.read_csv(os.path.join(data_dir, 'FG_flue_gas_properties_1700K.csv'))
props = {col: gas_props[col].values[5] for col in gas_props.columns}

cp_mixture = props['cp [kJ/kg.K]']             # [kJ/kg-K]
gamma_mixture = props['gamma']                # [-]
R_mix = props['R [kJ/kg.K]']                  # [kJ/kg-K]

# Discretization
dn = 1000
span = np.linspace(0, 1, dn)
r_prime = np.linspace(r_mean - b1_new / 2, r_mean + b1_new / 2, dn)
U_R_1 = omega * r_prime
alpha0_re = np.full(dn, alfa0)

# Reference conditions
T_ref = np.full(dn, 298.15)
P_ref = np.full(dn, 1.0)  # bar

# Station 0 properties
T0_re = np.full(dn, T0)
P0_re = np.full(dn, P0)
P0t_re = np.full(dn, P0t)

# Entropy at station 0 (kJ/kg-K)
s0_re = cp_mixture * np.log(T0_re / T_ref) - R_mix * np.log(P0_re / P_ref)

# Plot entropy
plt.figure()
plt.plot(s0_re, span, 'b-', linewidth=1.5)
plt.grid(True)
plt.xlabel('Entropy $s_0$ (kJ/kg·K)')
plt.ylabel('Spanwise Location (0 = Hub, 1 = Tip)')
plt.title('Spanwise Entropy Distribution at Station 0')
plt.xlim([min(s0_re)*0.98, max(s0_re)*1.02])
plt.savefig('plots/RE1_station0_entropy.png', dpi=300)
#plt.close()

# Save initial profiles
pd.DataFrame({
    'r_prime': r_prime,
    'span': span,
    'U_R_1': U_R_1,
    's0_re': s0_re
}).to_csv('data/RE_station0_profiles.csv', index=False)

print("Part 1 complete: Initialization and Station 0 processed and saved.")

In [ ]:
# Radial Equilibrium - Part 2: Station 1 Setup and Secondary Losses

# Load previous results
station0_df = pd.read_csv('data/RE_station0_profiles.csv')
r_prime = station0_df['r_prime'].values
span = station0_df['span'].values
U_R_1 = station0_df['U_R_1'].values
s0_re = station0_df['s0_re'].values

# Retrieve needed values
V1tang = get_value('V1tang')
V1ax = get_value('V1ax')
T1t = get_value('T1t')
Y_total_stator = get_value('Y_total_stator')
Rho1_new = get_value('rho1')

Y_secondary = get_value('Y_secondary_calculated')

# Blended vortex assumption (a = 0.5)
a = 0.5
V1tang_re = V1tang * (a * r_prime / r_mean + (1 - a) * r_mean / r_prime)

# Mass flow rate at mean radius
mass_flow_in_1 = b1_new * Rho1_new * 2 * np.pi * r_mean * V1ax

# Initial guesses for profiles
V1ax_re = np.full_like(r_prime, V1ax)
T1t_re = np.full_like(r_prime, T1t)
V1_re = np.sqrt(V1ax_re**2 + V1tang_re**2)
alpha1_re = np.degrees(np.arctan2(V1tang_re, V1ax_re))
T1_re = T1t_re - (V1_re**2 / (2 * cp_mixture * 1000))
P1_re = P0 * (T1_re / T0) ** (gamma_mixture / (gamma_mixture - 1))
rho1_re = (P1_re * 1e5) / (R_mix * 1000 * T1_re)
M1_re = V1_re / np.sqrt(gamma_mixture * R_mix * 1000 * T1_re)

# Initial elemental mass flow
elemental_mass_flow_1_init = rho1_re * V1ax_re * 2 * np.pi * r_prime * b1_new

# Secondary loss profile
delta_sh = 0.35
delta_st = 0.35
epsilon_mid = 0.10
normalized_profile_secondary = secondaryLossProfile(span, delta_sh, delta_st, epsilon_mid)

# Plot secondary loss
plt.figure()
plt.plot(normalized_profile_secondary, span, 'r-', linewidth=1.8)
plt.grid(True)
plt.xlabel('Secondary Loss $Y_{secondary}$')
plt.ylabel('Spanwise Location (0 = Hub, 1 = Tip)')
plt.title('Spanwise Secondary Loss Distribution (Stator)')
plt.xlim([0, 1.05])
plt.ylim([0, 1])
plt.savefig('plots/RE2_secondary_loss_profile_stator.png', dpi=300)
#plt.close()

# Distribute Y_other using elemental mass flow
Y_other = Y_total_stator - Y_secondary
distributed_Y_other_stator, _, _ = scaleLossProfile(Y_other, np.ones_like(span), r_prime, elemental_mass_flow_1_init)

# Plot other loss
plt.figure()
plt.plot(distributed_Y_other_stator, span, 'b-', linewidth=1.8)
plt.grid(True)
plt.xlabel('Other Loss $Y_{other}$')
plt.ylabel('Spanwise Location (0 = Hub, 1 = Tip)')
plt.title('Spanwise Distribution of Other Losses (Stator)')
plt.xlim([0.98*distributed_Y_other_stator[0], 1.02*distributed_Y_other_stator[0]])
plt.ylim([0, 1])
plt.savefig('plots/RE3_other_loss_profile_stator.png', dpi=300)
#plt.close()

# Save to CSV
pd.DataFrame({
    'span': span,
    'V1tang_re': V1tang_re,
    'V1ax_re': V1ax_re,
    'T1t_re': T1t_re,
    'T1_re': T1_re,
    'P1_re': P1_re,
    'rho1_re': rho1_re,
    'secondary_loss': normalized_profile_secondary,
    'distributed_Y_other_stator': distributed_Y_other_stator
}).to_csv('data/RE_station1_initial_profiles.csv', index=False)

print("Part 2 complete: Station 1 setup and secondary loss distribution done.")

In [ ]:
# Radial Equilibrium - Part 3: Radial equilibrium solver for Station 1 with Loss scaling

# Load required data
station0_df = pd.read_csv('data/RE_station0_profiles.csv')
station1_df = pd.read_csv('data/RE_station1_initial_profiles.csv')

r_prime = station0_df['r_prime'].values
span = station0_df['span'].values
U_R_1 = station0_df['U_R_1'].values
s0_re = station0_df['s0_re'].values

# Load station 1 values
V1ax_re = station1_df['V1ax_re'].values
V1tang_re = station1_df['V1tang_re'].values
T1t_re = station1_df['T1t_re'].values
T1_re = station1_df['T1_re'].values
P1_re = station1_df['P1_re'].values
rho1_re = station1_df['rho1_re'].values
normalized_profile_secondary = station1_df['secondary_loss'].values
distributed_Y_other_stator = station1_df['distributed_Y_other_stator'].values

# Start iteration
err_mass = np.inf
iteration = 0
max_iter = 50
dn = len(r_prime)

while err_mass > 1e-8 and iteration < max_iter:
    iteration += 1

    elemental_mass_flow = rho1_re * V1ax_re * 2 * np.pi * r_prime * b1_new

    distributed_secondary_loss, _, _ = scaleLossProfile(
        Y_secondary * np.ones(dn), normalized_profile_secondary, r_prime, elemental_mass_flow
    )

    Y_tot_stator_re = distributed_Y_other_stator + distributed_secondary_loss
    P1t_re = (P0t + P1_re * Y_tot_stator_re) / (1 + Y_tot_stator_re)
    #P0t_re = np.full_like(P1t_re, P0t)

    s1_re = s0_re - R_mix * np.log(P1t_re / P0t_re)
    ds_dr = np.gradient(s1_re, r_prime)

    def interp(r, array):
        return np.interp(r, r_prime, array)

    def dVax_dr(r, Vax):
        return -(interp(r, T1_re) / Vax) * interp(r, ds_dr)

    sol = solve_ivp(dVax_dr, [r_prime[0], r_prime[-1]], [V1ax_re[0]], t_eval=r_prime)
    V1ax_re = sol.y[0]

    V1_re = np.sqrt(V1ax_re**2 + V1tang_re**2)
    T1_re = T1t_re - V1_re**2 / (2 * cp_mixture * 1000)
    P1_re = P1t_re / (T1t_re / T1_re) ** (gamma_mixture / (gamma_mixture - 1))
    rho1_re = (P1_re * 1e5) / (R_mix * 1000 * T1_re)

    d_r = r_prime[1] - r_prime[0]
    mass_flow_radial = rho1_re * V1ax_re * 2 * np.pi * r_prime * d_r
    mass_flow_sum = np.sum(mass_flow_radial)

    scale_factor = np.sqrt(mass_flow_in_1 / mass_flow_sum)
    V1ax_re *= scale_factor
    err_mass = abs(mass_flow_sum - mass_flow_in_1) / mass_flow_in_1

# Final output
pd.DataFrame({
    'span': span,
    'V1ax_re': V1ax_re,
    'V1tang_re': V1tang_re,
    'T1_re': T1_re,
    'P1_re': P1_re,
    'rho1_re': rho1_re,
    's1_re': s1_re,
    'Y_tot_stator_re': Y_tot_stator_re
}).to_csv('data/RE_station1_radial_equilibrium.csv', index=False)

print("Part 3 complete: Radial equilibrium solver for Station 1 converged in", iteration, "iterations.")

In [ ]:
# Radial Equilibrium - Part 4: Mass-Averaged Quantities at Station 1

# Load required profile data
df = pd.read_csv('data/RE_station1_radial_equilibrium.csv')
rho1_re = df['rho1_re'].values
V1ax_re = df['V1ax_re'].values
V1tang_re = df['V1tang_re'].values
T1_re = df['T1_re'].values
P1_re = df['P1_re'].values
s1_re = df['s1_re'].values
r_prime = pd.read_csv('data/RE_station0_profiles.csv')['r_prime'].values

d_r = r_prime[1] - r_prime[0]
mass_flow_elemental = rho1_re * V1ax_re * 2 * np.pi * r_prime * d_r
mass_flow_total = np.sum(mass_flow_elemental)

# Helper function
def mass_avg(var):
    return np.sum(var * mass_flow_elemental) / mass_flow_total

# Compute mass-averaged values
V1ax_avg = mass_avg(V1ax_re)
V1tang_avg = mass_avg(V1tang_re)
V1_avg = mass_avg(np.sqrt(V1ax_re**2 + V1tang_re**2))
alpha1_avg = mass_avg(np.degrees(np.arctan2(V1tang_re, V1ax_re)))
T1_avg = mass_avg(T1_re)
T1t_avg = mass_avg(T1t_re)
P1_avg = mass_avg(P1_re)
P1t_avg = mass_avg(P1t_re)
rho1_avg = mass_avg(rho1_re)
M1_avg = mass_avg(M1_re)
s1_avg = mass_avg(s1_re)

# Save to CSV
avg_data = {
    'V1ax_avg': V1ax_avg,
    'V1tang_avg': V1tang_avg,
    'V1_avg': V1_avg,
    'alpha1_avg': alpha1_avg,
    'T1_avg': T1_avg,
    'T1t_avg': T1t_avg,
    'P1_avg': P1_avg,
    's1_avg': s1_avg,
    'mass_flow_total': mass_flow_total
}
pd.DataFrame([avg_data]).to_csv('data/RE_station1_mass_averaged.csv', index=False)


# Plot V1tang
plt.figure()
plt.plot(V1tang_re, span, 'b-', linewidth=1.5)
plt.grid(True)
plt.xlabel(r'Tangential Velocity $V_{\theta, 1}(r)$ [m/s]')
plt.ylabel('Spanwise Location (0 = Hub, 1 = Tip)')
plt.title('Blended Vortex Tangential Velocity Profile at Station 1')
plt.xlim([min(V1tang_re)*0.98, max(V1tang_re)*1.02])
plt.savefig('plots/RE4_V1tang_station1.png', dpi=300)
#plt.close()

# Plot V1ax
plt.figure()
plt.plot(V1ax_re, span, 'b-', linewidth=1.5)
plt.grid(True)
plt.xlabel('Axial Velocity $V_{ax, 1}(r)$ [m/s]')
plt.ylabel('Spanwise Location (0 = Hub, 1 = Tip)')
plt.title('Spanwise Axial Velocity Profile at Station 1 $(V_{ax,1})$')
# Force x-axis to show actual values, no offset or scientific notation
ax = plt.gca()
formatter = ScalarFormatter(useMathText=False, useOffset=False)
formatter.set_scientific(False)
ax.xaxis.set_major_formatter(formatter)
plt.savefig('plots/RE7_V1ax_station1.png', dpi=300)
#plt.close()


# Plot: Distributed Secondary Loss
plt.figure()
plt.plot(distributed_secondary_loss, span, 'b-', linewidth=1.5)
plt.grid(True)
plt.xlabel('Distributed Secondary Loss $Y_{secondary}$')
plt.ylabel('Spanwise Location (0 = Hub, 1 = Tip)')
plt.title('Final Spanwise Secondary Loss Distribution')
plt.xlim([0, 0.15])
plt.savefig('plots/RE5_secondary_loss_station1.png', dpi=300)
#plt.close()

# Plot: Total Loss Profile at Station 1
plt.figure()
plt.plot(Y_tot_stator_re, span, 'b-', linewidth=1.5)
plt.grid(True)
plt.xlabel('Total Loss $Y_{total}$')
plt.ylabel('Spanwise Location (0 = Hub, 1 = Tip)')
plt.title('Final Total Loss Profile Across Span at Station 1')
plt.xlim([0, 0.15])
plt.savefig('plots/RE6_total_loss_station1.png', dpi=300)
#plt.close()

# PLot: Entropy Profile
plt.figure()
plt.plot(s1_re, span, 'b-', linewidth=1.5)
plt.grid(True)
plt.xlabel('Entropy $s_1(r)$ [KJ/Kg K]')
plt.ylabel('Spanwise Location (0 = Hub, 1 = Tip)')
plt.title('Spanwise Entropy distribution at Station 1')
#plt.xlim([min(s1_re)*0.98, max(s1_re)*1.02])
plt.savefig('plots/RE8_entropy_station1.png', dpi=300)
#plt.close()


print("Part 4 complete: Mass-averaged quantities for Station 1 saved.")

In [ ]:
# Radial Equilibrium - Part 5: Rotor Outlet (Station 2) Initialization and Loss Setup

# Load Station 1 data
station1_df = pd.read_csv('data/RE_station1_radial_equilibrium.csv')
rho1_re = station1_df['rho1_re'].values
V1tang_re = station1_df['V1tang_re'].values
T1_re = station1_df['T1_re'].values
P1_re = station1_df['P1_re'].values
s1_re = station1_df['s1_re'].values

# Load geometry and span
r_prime = pd.read_csv('data/RE_station0_profiles.csv')['r_prime'].values
span = pd.read_csv('data/RE_station0_profiles.csv')['span'].values
U_R_1 = pd.read_csv('data/RE_station0_profiles.csv')['U_R_1'].values

dn = len(span)

# Load station2 results from turbine meanline export
station2_df = pd.read_csv('data/TM_station2_results.csv', index_col=0).squeeze()
station2_losses = pd.read_csv('data/TM_station2_rotor_losses.csv', index_col=0).squeeze()

V2tang = station2_df['V2tang']
V2ax = station2_df['V2ax']
T2t = station2_df['T2t']
b2_final = station2_losses['b2']
rho2_final = station2_losses['rho2']
Y_total_rotor = station2_losses['Y_total_rotor']
Y_secondary_rotor = station2_losses['Y_secondary']
Y_TC = station2_losses['Y_TC']

# Blended vortex for rotor outlet
a = 0.5
V2tang_re = V2tang * (a * r_prime / r_mean + (1 - a) * r_mean / r_prime)
mass_flow_in_2 = b2_final * rho2_final * 2 * np.pi * r_mean * V2ax

V2ax_re = np.full_like(r_prime, V2ax)
T2t_re = np.full_like(r_prime, T2t)
T2trel_re = T2t_re + U_R_1**2 / (2 * cp_mixture * 1000)
V2_re = np.sqrt(V2ax_re**2 + V2tang_re**2)
T2_re = T2t_re - V2_re**2 / (2 * cp_mixture * 1000)
P2_re = P1_re * (T2_re / T1_re)**(gamma_mixture / (gamma_mixture - 1))
rho2_re = (P2_re * 1e5) / (R_mix * 1000 * T2_re)
M2_re = V2_re / np.sqrt(gamma_mixture * R_mix * 1000 * T2_re)
alpha2_re = np.degrees(np.arctan(V2tang_re / V2ax_re))

# Initial elemental mass flow
elemental_mass_flow_2_init = rho2_re * V2ax_re * 2 * np.pi * r_prime * b2_final

# Loss profiles
normalized_profile_secondary_2 = secondaryLossProfile(span, 0.35, 0.35, 0.1)
normalized_profile_Tip_Clearance = tipClearanceLossProfile(span, 0.25, 'gaussian')
Y_other_2 = Y_total_rotor - Y_secondary_rotor - Y_TC
distributed_Y_other_2, _, _ = scaleLossProfile(Y_other_2, np.ones(dn), r_prime, elemental_mass_flow_2_init)

# Save initial Station 2 data
pd.DataFrame({
    'span': span,
    'r_prime': r_prime,
    'V2ax_re': V2ax_re,
    'V2tang_re': V2tang_re,
    'T2t_re': T2t_re,
    'T2trel_re': T2trel_re,
    'V2_re': V2_re,
    'T2_re': T2_re,
    'P2_re': P2_re,
    'rho2_re': rho2_re,
    'secondary_profile': normalized_profile_secondary_2,
    'tip_clearance_profile': normalized_profile_Tip_Clearance,
    'distributed_Y_other_2': distributed_Y_other_2
}).to_csv('data/RE_station2_initial_profiles.csv', index=False)

print("Part 5 complete: Rotor outlet (Station 2) initialized and loss profiles saved.")

In [ ]:
# Radial Equilibrium - Part 6: Station 2 Iterative Radial Equilibrium Solver

# Load Station 2 initial profile
df2 = pd.read_csv('data/RE_station2_initial_profiles.csv')
span = df2['span'].values
r_prime = df2['r_prime'].values
U_R_1 = pd.read_csv('data/RE_station0_profiles.csv')['U_R_1'].values

# Inputs from station 2
V2ax_re = df2['V2ax_re'].values
V2tang_re = df2['V2tang_re'].values
T2t_re = df2['T2t_re'].values
T2trel_re = df2['T2trel_re'].values
V2_re = df2['V2_re'].values
T2_re = df2['T2_re'].values
P2_re = df2['P2_re'].values
rho2_re = df2['rho2_re'].values
normalized_profile_secondary_2 = df2['secondary_profile'].values
normalized_profile_Tip_Clearance = df2['tip_clearance_profile'].values
distributed_Y_other_2 = df2['distributed_Y_other_2'].values

# Load values
P1trel_re = pd.read_csv('data/RE_station1_radial_equilibrium.csv')['P1_re'].values * (1 + Y_secondary_rotor)
P1t_re = P1trel_re / (1 + Y_secondary_rotor)
T1t_re = np.full_like(P1trel_re, get_value('T1t'))
s1_re = pd.read_csv('data/RE_station1_radial_equilibrium.csv')['s1_re'].values

# Iteration loop for station 2
dn = len(r_prime)
err_mass_2 = np.inf
iteration_2 = 0
max_iter = 1000

while err_mass_2 > 1e-8 and iteration_2 < max_iter:
    iteration_2 += 1

    elemental_mass_flow_2 = rho2_re * V2ax_re * 2 * np.pi * r_prime * b2_final

    distributed_secondary_loss_2, _, _ = scaleLossProfile(Y_secondary_rotor * np.ones(dn), normalized_profile_secondary_2, r_prime, elemental_mass_flow_2)

    distributed_tip_loss_2, _, _ = scaleLossProfile(Y_TC * np.ones(dn), normalized_profile_Tip_Clearance, r_prime, elemental_mass_flow_2)

    Y_tot_rotor_re = distributed_Y_other_2 + distributed_secondary_loss_2 + distributed_tip_loss_2

    P2trel_re = (P1trel_re + P2_re * Y_tot_rotor_re) / (1 + Y_tot_rotor_re)
    P2t_re = P2trel_re * (T2t_re / T2trel_re) ** (gamma_mixture / (gamma_mixture - 1))

    s2_re = s1_re + cp_mixture * np.log(T2t_re / T1t_re) - R_mix * np.log(P2t_re / P1t_re)
    ds_dr_2 = np.gradient(s2_re, r_prime)

    # Precompute gradients and interpolators
    Euler_work = U_R_1 * (V1tang_re - V2tang_re) / dn

    # Gradient of Euler work across span
    dEuler_dr = np.gradient(Euler_work, r_prime)

    # Create interpolator functions (with extrapolation enabled)
    interp_T2 = interp1d(r_prime, T2_re, kind='linear', fill_value='extrapolate', bounds_error=False)
    interp_dsdr_2 = interp1d(r_prime, ds_dr_2, kind='linear', fill_value='extrapolate', bounds_error=False)
    interp_dEulerdr = interp1d(r_prime, dEuler_dr, kind='linear', fill_value='extrapolate', bounds_error=False)

    # Final NISRE differential equation
    def dVax_dr_2(r, Vax):
        return (1 / (Vax * cp_mixture * 1000)) * interp_dEulerdr(r) - \
            (interp_T2(r) / Vax) * interp_dsdr_2(r)

    # Integrate with solve_ivp
    Vax_initial_2 = V2ax_re[0]
    sol2 = solve_ivp(dVax_dr_2, [r_prime[0], r_prime[-1]], [Vax_initial_2], t_eval=r_prime, method='RADAU')
    V2ax_re = sol2.y[0]

    # Euler work
    Euler_local = U_R_1 * (V1tang_re - V2tang_re)
    Euler_energy_flow = Euler_local * elemental_mass_flow_2
    total_Euler_energy = np.sum(Euler_energy_flow)
    total_mass_flow = np.sum(elemental_mass_flow_2)
    delta_h_Euler = total_Euler_energy / total_mass_flow / 1000

    # Physical Model Update
    print(np.average(s2_re-s1_re))
    T2t_re = T1t_re * np.exp((s2_re - s1_re) / cp_mixture) * np.exp(-delta_h_Euler / (cp_mixture * T1t_re))

    # Update properties
    V2_re = np.sqrt(V2ax_re**2 + V2tang_re**2)
    alpha2_re = np.degrees(np.arctan(V2tang_re / V2ax_re))
    T2_re = T2t_re - V2_re**2 / (2 * cp_mixture * 1000)
    #print(np.average(T2_re))
    P2_re = P2t_re / (T2t_re / T2_re) ** (gamma_mixture / (gamma_mixture - 1))
    rho2_re = (P2_re * 1e5) / (R_mix * 1000 * T2_re)

    M2_re = V2_re / np.sqrt(gamma_mixture * R_mix * 1000 * T2_re)
    W2ax_re = V2ax_re
    W2tang_re = V2tang_re - U_R_1
    W2_re = np.sqrt(W2ax_re**2 + W2tang_re**2)
    beta2_re = np.degrees(np.arctan(W2tang_re / W2ax_re))

    # Mass flow correction
    d_r_2 = r_prime[1] - r_prime[0]
    mass_flow_radial_2 = rho2_re * V2ax_re * 2 * np.pi * r_prime * d_r_2
    mass_flow_sum_2 = np.sum(mass_flow_radial_2)

    scale_factor_2 = np.sqrt(mass_flow_in_2 / mass_flow_sum_2)
    V2ax_re *= scale_factor_2
    err_mass_2 = np.absolute(mass_flow_sum_2 - mass_flow_in_2) / mass_flow_in_2

    print(f"Iteration {iteration_2} | V2ax_avg: {np.mean(V2ax_re):.2f} | T2_re_avg: {np.mean(T2_re):.2f} | rho2_min: {np.min(rho2_re):.5f}")


# Save results
pd.DataFrame({
    'span': span,
    'V2ax_re': V2ax_re,
    'V2tang_re': V2tang_re,
    'T2_re': T2_re,
    'P2_re': P2_re,
    'rho2_re': rho2_re,
    's2_re': s2_re,
    'Y_tot_rotor_re': Y_tot_rotor_re
}).to_csv('data/RE_station2_radial_equilibrium.csv', index=False)

print("Part 6 complete: Station 2 radial equilibrium converged in", iteration_2, "iterations.")

## Plotting

In [ ]:
# Radial Equilibrium - Part 7: Mass-Averaged Quantities at Station 2

# Load required profile data
df2 = pd.read_csv('data/RE_station2_radial_equilibrium.csv')
rho2_re = df2['rho2_re'].values
V2ax_re = df2['V2ax_re'].values
V2tang_re = df2['V2tang_re'].values
T2_re = df2['T2_re'].values
P2_re = df2['P2_re'].values
s2_re = df2['s2_re'].values
r_prime = pd.read_csv('data/RE_station0_profiles.csv')['r_prime'].values

d_r = r_prime[1] - r_prime[0]
mass_flow_elemental = rho2_re * V2ax_re * 2 * np.pi * r_prime * d_r_2
mass_flow_total = np.sum(mass_flow_elemental)

# Helper function
def mass_avg2(var):
    return np.sum(var * mass_flow_elemental) / mass_flow_total

# Compute mass-averaged values
V2ax_avg = mass_avg2(V2ax_re)
V2tang_avg = mass_avg2(V2tang_re)
V2_avg = mass_avg2(np.sqrt(V2ax_re**2 + V2tang_re**2))
alpha2_avg = mass_avg2(np.degrees(np.arctan2(V2tang_re, V2ax_re)))
T2_avg = mass_avg2(T2_re)
P2_avg = mass_avg2(P2_re)
s2_avg = mass_avg2(s2_re)

# Save to CSV
avg_data = {
    'V2ax_avg': V2ax_avg,
    'V2tang_avg': V2tang_avg,
    'V2_avg': V2_avg,
    'alpha2_avg': alpha2_avg,
    'T2_avg': T2_avg,
    'P2_avg': P2_avg,
    's2_avg': s2_avg,
    'mass_flow_total': mass_flow_total
}
pd.DataFrame([avg_data]).to_csv('data/RE_station2_mass_averaged.csv', index=False)

# LOSSES PLOTS
# Plot: Normalized Secondary loss Profile at Rotor Outlet
plt.figure()
plt.plot(normalized_profile_secondary_2, span, 'b-', linewidth=1.5)
plt.grid(True)
plt.xlabel('Normalized Secondary Loss Profile')
plt.ylabel('Spanwise Location (0 = Hub, 1 = Tip)')
plt.title('Rotor Outlet - Normalized Secondary Loss Profile')
plt.savefig('plots/RE9_NormalizedSecondaryLoss_RotorOutlet.png', dpi=300)
#plt.close()

# Plot: Normalized Tip Clearance Loss Profile at Rotor Outlet
plt.figure()
plt.plot(normalized_profile_Tip_Clearance, span, 'b-', linewidth=1.5)
plt.grid(True)
plt.xlabel('Normalized Tip Clearance Loss Profile')
plt.ylabel('Spanwise Location (0 = Hub, 1 = Tip)')
plt.title('Rotor Outlet - Normalized Tip Clearance Loss Profile')
plt.savefig('plots/RE10_NormalizedTipClearanceLoss_RotorOutlet.png', dpi=300)
#plt.close()

# Plot: Distributed Other Losses at Rotor Outlet
plt.figure()
plt.plot(distributed_Y_other_2, span, 'b-', linewidth=1.5)
plt.grid(True)
plt.xlabel('Distributed $Y_{other} (r)$')
plt.ylabel('Spanwise Location (0 = Hub, 1 = Tip)')
plt.title('Rotor Outlet - Distributed Other Losses')
plt.savefig('plots/RE11_DistributedOtherLosses_RotorOutlet.png', dpi=300)
#plt.close()

# Plot: Distributed Secondary Losses at Rotor Outlet
plt.figure()
plt.plot(distributed_secondary_loss_2, span, 'b-', linewidth=1.5)
plt.grid(True)
plt.xlabel('Distributed Secondary Loss $Y_{secondary} (r)$')
plt.ylabel('Spanwise Location (0 = Hub, 1 = Tip)')
plt.title('Rotor Outlet - Distributed Secondary Losses')
plt.savefig('plots/RE12_DistributedSecondaryLosses_RotorOutlet.png', dpi=300)
#plt.close()

# Plot: Distributed Tip Clearance Losses at Rotor Outlet
plt.figure()
plt.plot(distributed_tip_loss_2, span, 'b-', linewidth=1.5)
plt.grid(True)
plt.xlabel('Distributed Tip Clearance Loss $Y_{TC} (r)$')
plt.ylabel('Spanwise Location (0 = Hub, 1 = Tip)')
plt.title('Rotor Outlet - Distributed Tip Clearance Losses')
plt.savefig('plots/RE13_DistributedTipLosses_RotorOutlet.png', dpi=300)
#plt.close()

# Plot: Distributed TOTAL Losses at Rotor Outlet
plt.figure()
plt.plot(Y_tot_rotor_re, span, 'b-', linewidth=1.5)
plt.grid(True)
plt.xlabel('Total Loss $Y_{Total} (r)$')
plt.ylabel('Spanwise Location (0 = Hub, 1 = Tip)')
plt.title('Rotor Outlet - Total Losses Distribution')
plt.savefig('plots/RE14_DistributedTOTAL_RotorOutlet.png', dpi=300)
#plt.close()


# OTHER PLOTS
# Plot: Entropy distribution at Rotor Outlet
plt.figure()
plt.plot(s2_re, span, 'b-', linewidth=1.5)
plt.grid(True)
plt.xlabel('Entropy $s_2(r)$ [kJ/kg K]')
plt.ylabel('Spanwise Location (0 = Hub, 1 = Tip)')
plt.title('Rotor Outlet - Entropy Distribution')
plt.savefig('plots/RE15_Entropy_RotorOutlet.png', dpi=300)
#plt.close()

# Plot: Entropy distribution at Rotor Outlet
plt.figure()
plt.plot(V2ax_re, span, 'b-', linewidth=1.5)
plt.xlabel('Axial Velocity $V_{ax, 2}(r)$ [m/s]')
plt.ylabel('Spanwise Location (0 = Hub, 1 = Tip)')
plt.title('Spanwise Axial Velocity Profile at Rotor Outlet $(V_{ax, 2})$')
# Force x-axis to show actual values, no offset or scientific notation
ax = plt.gca()
formatter = ScalarFormatter(useMathText=False, useOffset=False)
formatter.set_scientific(False)
ax.xaxis.set_major_formatter(formatter)
plt.grid(True)
plt.savefig('plots/RE16_V2ax_RotorOutlet.png', dpi=300)
#plt.close()

# Plot: Entropy distribution at Rotor Outlet
plt.figure()
plt.plot(T2t_re, span, 'b-', linewidth=1.5)
plt.grid(True)
plt.xlabel('Total Temperature $T_{0,2}(r)$ [K]')
plt.ylabel('Spanwise Location (0 = Hub, 1 = Tip)')
plt.title('Rotor Outlet - Total Temperature Distribution')
plt.savefig('plots/RE17_T2_RotorOutlet.png', dpi=300)
#plt.close()


print("Part 7 complete: Mass-averaged quantities for Station 2 saved.")

## Blade Design

# Compressor

## Input Data (matching the Turbine) and Assumptions

## Combustor Balance

## Power Balance